## Import

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

from sklearn.base import BaseEstimator, TransformerMixin
import re
import numpy as np
import nlpaug.augmenter.word as naw # To synonym, see  to more info https://github.com/makcedward/nlpaug
import nlpaug.augmenter.char as nac
from mtranslate import translate
import pickle
from gensim.models.keyedvectors import KeyedVectors
import os
from nltk.tokenize import TweetTokenizer

## Data

In [3]:
# handmade
hand_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_nawel.csv", sep = ';')
hand_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_nawel.csv", sep = ';')
hand_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_nawel.csv", sep = ';')
hand_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_nawel.csv", sep = ';')

hand_tavo_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_tavo.csv", sep = ';')
hand_tavo_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_tavo.csv", sep = ';')
hand_tavo_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_tavo.csv", sep = ';')
hand_tavo_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_tavo.csv", sep = ';')

hand_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_martin.csv", sep = ';', encoding = 'latin-1')
hand_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_martin.csv", sep = ';')
hand_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_martin.csv", sep = ';')
hand_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_martin.csv", sep = ';')

In [4]:
df_ = pd.concat([hand_nawel_0, hand_nawel_1, hand_nawel_2, hand_nawel_3,
                 hand_tavo_0, hand_tavo_1, hand_tavo_2, hand_tavo_3,
                 hand_martin_0, hand_martin_1, hand_martin_2, hand_martin_3])

df_.count()

class    345
text     345
dtype: int64

In [5]:
df_['class'].value_counts()

class
0    101
1     89
2     87
3     68
Name: count, dtype: int64

## Trabajando con la data

In [9]:
# To vocabulary
!python -m spacy download es

                                              0.0/12.9 MB ? eta -:--:--
                                              0.1/12.9 MB 2.6 MB/s eta 0:00:05
     ---                                      1.0/12.9 MB 13.0 MB/s eta 0:00:01
     ----                                     1.3/12.9 MB 10.5 MB/s eta 0:00:02
     --------                                 2.6/12.9 MB 15.3 MB/s eta 0:00:01
     ---------                                3.1/12.9 MB 16.4 MB/s eta 0:00:01
     -------------                            4.5/12.9 MB 16.9 MB/s eta 0:00:01
     ---------------------                    6.9/12.9 MB 22.2 MB/s eta 0:00:01
     -----------------------                  7.5/12.9 MB 20.9 MB/s eta 0:00:01
     -----------------------------            9.4/12.9 MB 23.2 MB/s eta 0:00:01
     ----------------------------------      11.3/12.9 MB 32.8 MB/s eta 0:00:01
     -------------------------------------   12.5/12.9 MB 31.1 MB/s eta 0:00:01
     --------------------------------------  12.9

In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [69]:
# Creation Of tuples (class, text)
dataset = [
    (row['class'], row['text'].lower()) # With lower
    for index, row in df_.iterrows()
]

from random import sample
for example in sample(dataset, 5):
    print("Example:\n", example)

Example:
 (2, ' veamos este tema otro dia por favor')
Example:
 (0, 'voy a llegar a la hora ')
Example:
 (0, 'lo tengo agendado y confirmado')
Example:
 (1, 'se me complicó')
Example:
 (2, 'me re-agenda la cita que teniamos a cualquier hora el sabado')


In [74]:
# Pum ahora hagamos la arquitectura
# simplecita, un capa de embedding, y luego una red feed forward de
import torch.nn as nn
import torch.nn.functional as F

# Red neuronal con una sola capa oculta
class ArgumentClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, cnn_pool_channels = 24, cnn_kernel_size = 3):#, hidden_size, pad_idx):
        super().__init__()

        # capa de embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, mode="mean")

        # capas de la MLP
        #self.fc = nn.Linear(embed_dim, num_class)
        # self.fc1 = nn.Linear(embed_dim, hidden_size)
        # self.fc2 = nn.Linear(hidden_size, num_class)
        self.conv = nn.Conv1d(
            in_channels=1,
            out_channels=cnn_pool_channels,
            kernel_size=cnn_kernel_size * embed_dim,
            stride=embed_dim,
        )

        # Calculamos el tamaño de entrada de la capa lineal
        fc_in_size = cnn_pool_channels

        # Creamos la capa lineal
        self.fc = nn.Linear(fc_in_size, num_classes)

        # Inicializamos los pesos de las capas
        self.init_weights()

    def init_weights(self):
        # Definimos el rango de los valores iniciales de los pesos
        initrange = 0.5

        # Inicializamos los pesos de la capa de embedding
        self.embedding.weight.data.uniform_(-initrange, initrange)

        # Inicializamos los pesos de la capa lineal
        self.fc.weight.data.uniform_(-initrange, initrange)

        # Inicializamos los sesgos de la capa lineal en cero
        self.fc.bias.data.zero_()

    def forward(self, batch):
         # Preparamos el input de la capa de embeddings a partir de text y offsets
        padded_inputs, _, _ = batch
        padded_inputs = torch.tensor(padded_inputs) 
        h = self.embedding(padded_inputs)
        h = h.view(h.size(0), 1, -1)
        h = torch.relu(self.conv(h))
        h = h.mean(dim=2)
        output = self.fc(h)
        return F.log_softmax(output, dim=1)

In [75]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("spacy", "es")
min_freq = 1
vocab = build_vocab_from_iterator((tokenizer(text[1]) for text in train_split), min_freq=min_freq)
labels = list({doc[0] for doc in train_split})
label_map = {label: index for index, label in enumerate(labels)}

UNK_IDX = 0
vocab.set_default_index(UNK_IDX)

vocab.insert_token('<pad>', 1)

stoi = vocab.get_stoi()

print("Vocabulary len:", len(vocab))
print("Random Words:", sample(vocab.get_itos(), 5))
print("Label's number:", len(labels))

Vocabulary len: 436
Random Words: ['atiendan', 'asistiré', 'wenaa', '<pad>', '27']
Label's number: 4


In [76]:
# Assuming you have the required parameters for the model
vocab_size = len(vocab)
embed_dim = 32
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create instances of the custom Dataset class
train_dataset = CustomDataset(train_data, tokenizer)
test_dataset = CustomDataset(test_data, tokenizer)

# Define data loaders with collate_fn
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

model = ArgumentClassifier(vocab_size, embed_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [77]:

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels, _ in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels, _ in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss += criterion(outputs, labels).item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Test Loss: {test_loss/len(test_loader):.4f}, Accuracy: {(100 * correct / total):.2f}%")

TypeError: expected Tensor as element 0 in argument 0, but got list